In [23]:
from pathlib import Path
from vgio.quake.bsp import Bsp
import pyvista as pv
import numpy as np
from dotdict import dotdict

quake_root = Path("D:\\Games\\quake")

# pv.set_jupyter_backend('ipyvtklink')  
pv.set_jupyter_backend('pythreejs')  
# pv.set_jupyter_backend('static')  

content_types = {}
content_types[-1]=dotdict({"contents":"CONTENTS_EMPTY"})
content_types[-2]=dotdict({"contents":"CONTENTS_SOLID"})
content_types[-3]=dotdict({"contents":"CONTENTS_WATER"})
content_types[-4]=dotdict({"contents":"CONTENTS_SLIME"})
content_types[-5]=dotdict({"contents":"CONTENTS_LAVA"})
content_types[-6]=dotdict({"contents":"CONTENTS_SKY"})

# custom clamp
clamp = lambda x, l, u: l if x < l else u if x > u else x

def lerp(a,b,t):
   return a+(b-a)*t

def v_lerp(a,b,t):
   return [
      lerp(a[0],b[0],t),
      lerp(a[1],b[1],t),
      lerp(a[2],b[2],t)]

def plane_dot(plane,p):
   n = plane.normal
   return (n[0]*p[0] + n[1]*p[1] + n[2]*p[2], plane.distance)

def classify_point(plane, p):
   n = plane.normal
   return int(n[0]*p[0] + n[1]*p[1] + n[2]*p[2] < plane.distance)

def bsp_locate(node,pos):
   tabs = 0
   while not node.contents:
      n = node.plane.normal
      side = classify_point(node.plane,pos)
      # print(f"{' '*tabs}{n[0]} {n[1]} {n[2]} > {node.plane.distance}: {side}")
      node = node.children[side]
   return node

def bsp_ray_collect(node,p0,p1,t0,t1,out):
   if node.contents: return

   dist,node_dist=plane_dot(node.plane,p0)
   otherdist,_=plane_dot(node.plane,p1)
   side,otherside=int(dist<node_dist),int(otherdist<node_dist)
   if side==otherside:
      # go down this side
      return bsp_ray_collect(node.children[side],p0,p1,t0,t1,out)

   # crossing a node
   t=dist-node_dist
   if t<0:
      t=t+0.03125
   else:
      t=t-0.03125

   # cliping fraction
   frac=clamp(t/(dist-otherdist),0,1)
   tmid,pmid=lerp(t0,t1,frac),v_lerp(p0,p1,frac)

   is_solid = bsp_locate(node.children[side],pmid).contents!="CONTENTS_SOLID"
   is_otherside_solid = bsp_locate(node.children[otherside],pmid).contents!="CONTENTS_SOLID"
   if is_solid and not is_otherside_solid:
      scale=1 if side==0 else -1
      n=node.plane.normal
      out.append(dotdict({
         "n" : [scale*n[0],scale*n[1],scale*n[2]],
         "t" : tmid,
         "pos": pmid
      }))

   if is_otherside_solid and not is_solid:
      scale=1 if otherside==0 else -1
      n=node.plane.normal
      out.append(dotdict({
         "n" : [scale*n[0],scale*n[1],scale*n[2]],
         "t" : tmid,
         "pos": pmid
      }))

   # go down both side
   bsp_ray_collect(node.children[side],p0,pmid,t0,tmid,out)
   bsp_ray_collect(node.children[int(not side)],pmid,p1,tmid,t1,out)

def bsp_ray_intersect(node,p0,p1,t0,t1,out):
    contents=node.contents  
    if contents:
        # is "solid" space (bsp)
        if contents!=-2:
            out.all_solid = False
            if contents==-1:
                out.in_open = True
            else:
                out.in_water = True
        else:
            out.start_solid = True
        # empty space
        return True

    dist,node_dist=plane_dot(node.plane,p0)
    otherdist,_=plane_dot(node.plane,p1)
    side,otherside=dist>node_dist,otherdist>node_dist
    if side==otherside:
        # go down this side
        return bsp_ray_intersect(node.children[0 if side else 1],p0,p1,t0,t1,out)

    # crossing a node
    t=dist-node_dist
    if t<0:
        t=t+0.03125
    else:
        t=t-0.03125

    # cliping fraction
    frac=clamp(t/(dist-otherdist),0,1)
    tmid,pmid=lerp(t0,t1,frac),v_lerp(p0,p1,frac)
    if not bsp_ray_intersect(node.children[0 if side else 1],p0,pmid,t0,tmid,out):
        return False

    child_id = 0 if not side else 1
    if bsp_locate(node.children[child_id],pmid).contents != "CONTENTS_SOLID":
        return bsp_ray_intersect(node.children[child_id],pmid,p1,tmid,t1,out)

    # never got out of the solid area
    if out.all_solid:
        return False

    scale=1 if side else -1
    n=node.plane.normal
    out.n = [scale*n[0],scale*n[1],scale*n[2]]
    out.t = tmid
    out.pos = pmid
    return False

def read_bsp(name):
   with Bsp.open(f"D:\\Games\\quake\\id1\\maps\\{name}.bsp") as bsp_file:
      planes = bsp_file.planes
      meshes = bsp_file.meshes()            
      clipnodes = bsp_file.clip_nodes

      # main model
      model = bsp_file.models[0]

      hulls = []
      for node in clipnodes:
         hulls.append(dotdict({
            "plane": planes[node.plane_number],
            "children": [node.children[0], node.children[1]]
         }))
      for node in hulls:
         def attach_node(side):
            children = node.children
            id = children[side] 
            if id<0:
               children[side] = content_types[id]
            else:
               children[side] = hulls[id]
         attach_node(0)
         attach_node(1)
      # display 32 unit bsp
      def dump_hull(node):
         content = node.get("content", None)
         if content:
            print(content)
         else:
            children = node.get("children")
            dump_hull(children[0])
            dump_hull(children[1])
      # dump_hull(hulls[bsp_file.models[0].head_node[1]])

      faces = bsp_file.faces[model.first_face:model.first_face + model.number_of_faces]
      tris = []
      for face in faces:
         edges = bsp_file.surf_edges[face.first_edge:face.first_edge + face.number_of_edges]
         verts = []
         for edge_id in edges:
            v = bsp_file.edges[abs(edge_id)].vertexes
            v0,v1 = v if edge_id<0 else reversed(v)
            verts.append(v0)
         # Ignore degenerate faces
         if len(verts) < 3:
            continue
            
         verts.insert(0,len(verts))         
         tris.append(verts)
         
      return pv.PolyData(np.array([[v.x,v.y,v.z] for v in bsp_file.vertexes]), np.hstack(tris)), model, hulls

pdata, model, hulls = read_bsp("aerowalk")
pl = pv.Plotter()
# base geometry
pl.add_mesh(pdata, style='wireframe', lighting=False)

bmin = model.bounding_box_min
bmax = model.bounding_box_max

# player=[-352,232,24+8]
# print(f"player: {bsp_locate(root_node,player)}")
total = 0
inside = []
vectors = []
# head_node 0 references *nodes* array, not clipnodes
root_node = hulls[model.head_node[1]]
for x in range(int(bmin[0]), int(bmax[0]+0.5), 8):
   for y in range(int(bmin[1]), int(bmax[1]+0.5), 8):
      out = []
      bsp_ray_collect(root_node,[x,y,bmin[2]],[x,y,bmax[2]],0,1,out)
      for hit in out:
         total += 1
         # walkable?
         if hit.n[2]>0.7:
            inside.append(hit.pos)
            vectors.append(hit.n)

vectors = np.array(vectors)
pempty = pv.PolyData(np.array(inside))
# add and scale
pempty["vectors"] = vectors * 0.3

arrows = pempty.glyph(
    orient='vectors',
    scale=False,
    factor=8,
)

pl.add_mesh(pempty, style='points', point_size=10.0, render_points_as_spheres=True, lighting=False)
pl.add_mesh(arrows, color='lightblue')

pl.show_grid()
pl.show()


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_d937fe1a-1eff-4f20-a98b-f1551e7d3797'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_4505053f-8e77-4af0-825a-16233f90eb48'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_d46175c2-7f03-4b37-98cf-10ef2a31f59d'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_39c61301-d244-4707-8696-9a90ec9bd6c3'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_6d7ecc64-bc48-4e7d-a708-4956ccecf7a5'.